In [82]:
import re
import json
import sys, glob, codecs
with open("fr/professions.txt", 'r') as f:
    prof = [line.rstrip('\n').strip() for line in f]
with open("fr/occupations.txt", 'r') as f:
    prof2 = [line.rstrip('\n').strip() for line in f]  
prof.extend(prof2)
prof=list(set(prof))
print len(prof)

254


In [72]:
from bs4 import BeautifulSoup
from wikitools import wiki, api
import json
import re


def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

    
def check_revision(revisions):
    redir=""
    page=False
    for j in revisions:
        if ("#REDIRECTION" in j["*"])|("#REDIRECT" in j["*"])|("#redirect" in j["*"])\
        |("#Redirect" in j["*"])|("#Redirection" in j["*"])|("#redirection" in j["*"]):
            m = re.search(r"\[\[(.+)\]\]", j["*"])
            s = m.group(0)[2:].split('#', 1)[0].replace("]]",'')
            redir=s
        elif len(j["*"])<=30:
            print j["*"]
            page=True
        else:
            page=True
    return redir,page

def check_wiki(beruf_m):
    global l
    site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
    #last revision
    params={'action':'query','titles':beruf_m,'prop':'revisions','rvprop':'timestamp|content','format':'json','continue':''}
    request = api.APIRequest(site, params)
    result=request.query()
    
    if result["query"]["pages"].values()[0].get("revisions"):
        redir,page=check_revision(result["query"]["pages"].values()[0]["revisions"])
        if redir:
            redirection_m[beruf_m]=redir
            return True
        if page:
            page_m.append(beruf_m)
            return True
    else:
        return False

redirection_m={}
page_m=[]
no_page_m=[]

other=[]  
l="fr" 

#=====progressbar
import sys #for progressbar
k=0#for the progressbar
number_pages=len(prof)#for the progressbar
steps = number_pages/50 #progressbar step (each 2% show)
#======

for i in prof:
    beruf_m=i.split("/")[0].split("[")[0].strip()
    beruf_m=re.sub("[*?\(\[].*?[\)\]]", "", beruf_m).strip()
    if check_wiki(beruf_m):
        pass
    else:
        if i!=beruf_m:
            #print i
            if ("(" in i)&(i[i.find("(")+1].isupper()):
                new_i=i[i.find("(")+1:].replace(")","")
                if "/" not in new_i:
                    if not check_wiki(new_i):
                        no_page_m.append(beruf_m)
                else:
                    other.append(new_i)
            elif ("/" in i)&(i[i.find("/")+2].isupper()):
                i_=i.split(" / ")
                k_m=False
                for j in i_:
                    if j!=beruf_m:
                        if check_wiki(j):
                            k_m=True
                if not k_m:
                    no_page_m.append(beruf_m)
                k_m=False
            else:
                no_page_m.append(beruf_m)
            #no_page_m.append(beruf_m)
        else:
            no_page_m.append(beruf_m)
        
    #=====progressbar 
    if steps!=0:
        if k%steps == 0:
            print '\b'+str((k)*100/number_pages)+'%',
            sys.stdout.flush()
    k=k+1#forthe  progressbar

                

0% 1% 3% 5% 7% 9% 11% 13% 15% 17% 19% 21% 23% 25% 27% 29% 31% 33% 35% 37% 39% 41% 43% 45% 47% 49% 51% 53% 55% 57% 59% 61% 62% 64% 66% 68% 70% 72% 74% 76% 78% 80% 82% 84% 86% 88% 90% 92% 94% 96% 98%


In [81]:
print len(no_page_m)
print len(page_m)
print len(redirection_m)
print len(no_page_m)+len(page_m)+ len(redirection_m)+len(other)#118#254
print len(other)

l="fr"

with open(l+'/wiki/m_page.json', 'w') as f:
    json.dump(page_m,  f,  indent=4)
with open(l+'/wiki/m_no_page.json', 'w') as f:
    json.dump(no_page_m,  f,  indent=4)
with open(l+'/wiki/m_redirection.json', 'w') as f:
    json.dump(redirection_m,  f,  indent=4)
with open(l+'/other.json', 'w') as f:
    json.dump(other,  f,  indent=4)
    


2


In [4]:
import json
from prettytable import PrettyTable

def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)


l="fr"
m_no_page=load_simple_json(l+'/wiki/m_no_page.json')
m_redirection=load_simple_json(l+'/wiki/m_redirection.json')
m_page=load_simple_json(l+'/wiki/m_page.json')

words=len(m_no_page)+len(m_page)+ len(m_redirection)


t = PrettyTable(["--{0}--".format(l),"Occup.", 'Wiki pages', "Redirects","No wiki page or redir"])
            #,"Ambigious pages"
t.add_row(['Masculine',words, len(m_page),  
           len(m_redirection),len(m_no_page)]) #,len(page_m_ambigious)

print t



+-----------+--------+------------+-----------+-----------------------+
|   --fr--  | Occup. | Wiki pages | Redirects | No wiki page or redir |
+-----------+--------+------------+-----------+-----------------------+
| Masculine |  252   |     92     |     47    |          113          |
+-----------+--------+------------+-----------+-----------------------+


# Store all from Category:Professions (fr: "Catégorie:Métier")

In [6]:
#http://fr.wikipedia.org/wiki/Catégorie:Métier

#store pages from category Beruf
from wikitools import wiki, api,category
import json

def seen_wiki_categories(lang,category_page,overloop=0,n=6):
    global seen_categories_
    global seen_pages_
    where_go_categories_=[]
    #print "We are here: "+ category_page
    seen_categories_.append(category_page)
    site = wiki.Wiki("http://"+lang+".wikipedia.org/w/api.php") 
    cat = category.Category(site, category_page) # Create object for "Category:xxxxx"
    for article in cat.getAllMembersGen(titleonly=True,namespaces=[0]): # iterate through all the pages in ns 0
        if article not in seen_pages_ :
            #if "," not in article:
            seen_pages_.append(article)
        else:
            continue
    for subcategory in cat.getAllMembersGen(titleonly=True,namespaces=[14]):
        if subcategory not in seen_categories_ :
            where_go_categories_.append(subcategory)
        else:
            continue
    for c in where_go_categories_ :
        if (c not in seen_categories_):
            if overloop<n:
                seen_wiki_categories(lang,c,overloop=overloop+1,n=n)
            else:
                print "overloop ==="+c
                pass

l="fr"
seen_categories_=[]
seen_pages_=[]
seen_wiki_categories(l,"Catégorie:Métier",overloop=0,n=2)
seen_pages_=list(set(seen_pages_))
print len(seen_pages_) #6627
with open(l+'/wiki/berufe_wiki.json', 'w') as f: 
    json.dump(seen_pages_,  f,  indent=4)#,ensure_ascii=False)




overloop ===Catégorie:Agriculteur
overloop ===Catégorie:Personnel forestier
overloop ===Catégorie:Métier de la vigne et du vin
overloop ===Catégorie:Gainier
overloop ===Catégorie:Métier équestre
overloop ===Catégorie:Animateur
overloop ===Catégorie:Métier artistique
overloop ===Catégorie:Métier de la musique
overloop ===Catégorie:Métier du cinéma
overloop ===Catégorie:Métier du théâtre
overloop ===Catégorie:Auteur de bande dessinée
overloop ===Catégorie:Éditeur de bande dessinée
overloop ===Catégorie:Coloriste de bande dessinée
overloop ===Catégorie:Dessinateur de bande dessinée
overloop ===Catégorie:Diffuseur de bande dessinée
overloop ===Catégorie:Encreur de bande dessinée
overloop ===Catégorie:Scénariste de bande dessinée
overloop ===Catégorie:Spécialiste de la bande dessinée
overloop ===Catégorie:Chauffage, ventilation et climatisation
overloop ===Catégorie:Électricien
overloop ===Catégorie:Fumisterie
overloop ===Catégorie:Maçonnerie
overloop ===Catégorie:Métallerie
overloop ===Cat

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:22: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:27: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


UnicodeEncodeError: 'ascii' codec can't encode character u'\xe9' in position 10: ordinal not in range(128)

In [7]:
with open(l+'/wiki/berufe_wiki.json', 'w') as f: 
    json.dump(seen_pages_,  f,  indent=4)#,ensure_ascii=False)

# Levenstein dictance

In [8]:
import json

def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)


l="fr"
m_no_page=load_simple_json(l+'/wiki/m_no_page.json')
m_redirection=load_simple_json(l+'/wiki/m_redirection.json')
m_page=load_simple_json(l+'/wiki/m_page.json')
wiki_pages=load_simple_json(l+'/wiki/berufe_wiki.json')


In [9]:
j=0
k=0
n=0
for i in wiki_pages:#632
    if i in m_page:
        j+=1
    elif i in m_redirection:
        k+=1
    elif i in m_no_page:
        n+=1
print j,k,n #58 from 92,8 from 47,0 

56 8 0


In [14]:
m=0
k=0
substr_m_wiki={}
substr_wiki_m={}
for i in m_no_page:
    for j in wiki_pages:
        if i in j:#if some m_no_page is a substring of berufe_wiki 
            m+=1
            print i,"==>",j
            if substr_m_wiki.has_key(i):
                substr_m_wiki[i].append(j)
            else:
                substr_m_wiki[i]=[j]
        if j in i: #if some berufe_wiki is a substring of m_no_page  
            k+=1 
            print i,"<==",j
            if substr_wiki_m.has_key(i):
                substr_wiki_m[i].append(j)
            else:
                substr_wiki_m[i]=[j]
            
print m,k 

Enseignant de la conduite de bâteaux de plaisance à moteur <== Enseignant
Officier chargé du quart à la passerelle <== Officier chargé du quart
Technicien en réseau de communication <== Technicien
Teinturier pressing <== Teinturier
Inspecteur des matériels de pulvérisation <== Inspecteur
Chocolatier confiseur <== Chocolatier
Photographe navigant professionnel <== Photographe
Expert-foncier, agricole et forestier <== Expert
Plâtrier-plaquiste <== Plâtrier
Menuisier-Agenceur <== Menuisier
Serrurier - Métallier <== Serrurier
Chauffeur de voiture de tourisme <== Chauffeur
Expert-automobile <== Expert
Bijoutier Joaillier <== Bijoutier
Monteur - Électricien <== Électricien
Monteur - Électricien <== Monteur
Technicien de laboratoire médical <== Technicien de laboratoire
Technicien de laboratoire médical <== Technicien
Bottier ==> Bottier orthopédiste
Enseignant de la conduite et de la sécurité routières <== Enseignant
Professionnel du pesage <== Profession
Carreleur - Mosaïste <== Mosaïste
Ca

In [11]:
#Levenshtein
import Levenshtein

print "With Levenshtein distance<=2 or ratio>0.8"
both6={}
s=0
r=0
d=0
for i in m_no_page:
    for j in wiki_pages: 
                    if i in j:
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
                        s+=1
                    elif Levenshtein.ratio(i, j)>0.8:
                        print "===>",i,"==",j
                        r+=1
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
                    elif Levenshtein.distance(i, j)<=2:
                        d+=1
                        print i,":",j    
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
print s,r,d


With Levenshtein distance<=2 or ratio>0.8
===> Accordeur de piano == Porteur de piano
Crêpier : Croupier
Bonnetier : Tonnelier
===> Personnel de chauffe == Personnel de hall
Layetier : Papetier
Layetier : Waretier
Layetier : Savetier
Façonnier : Wagonnier
===> Contrôleur technique de la construction == Contrôle technique de la construction en France
===> Oculariste == Oculiste
===> Technicien de laboratoire médical == Technicien de laboratoire
Bottier : Bouvier
Bottier : Blatier
===> Peintre en décor == Peintre de cour
1 6 8


In [16]:
print "======Masculine_no_pages in list of pages from category Beruf (substring inclusion):======"
for i in substr_m_wiki:
    for j in substr_m_wiki[i]:
        if j in m_page:
            print "!!!already in pages",j
        elif j in m_redirection:
            print "!!!links to smth",j
        elif j in m_redirection.values():
            print "!!!links to smth value",j
print "======Masculine_no_pages in list of pages from category Beruf (substring inclusion):======"            
for i in substr_wiki_m:        
    for j in substr_wiki_m[i]:
        if j in m_page:
            print "!!!already in pages",j
        elif j in m_redirection:
            print "!!!links to smth",j
        elif j in m_redirection.values():
            print "!!!links to smth value",j
            

======Masculine_no_pages in list of pages from category Beruf (substring inclusion):======
======Masculine_no_pages in list of pages from category Beruf (substring inclusion):======
!!!links to smth value Serrurier
!!!already in pages Mosaïste
!!!already in pages Photographe
!!!already in pages Électricien


In [18]:

def check_pages_in_list(both6):
    new_both6={}
    for i in both6:
        if i not in new_both6.keys():
            new_both6[i]=both6[i]
        for j in both6[i]:
            if j in m_page:
                index=int(both6[i].index(j))
                new_both6[i][index]=both6[i][index],1 #1=page exist
            elif j in m_redirection:
                index=int(both6[i].index(j))
                new_both6[i][index]=both6[i][index],3 #3=redirects to smth
            elif j in m_redirection.values():
                index=int(both6[i].index(j))
                new_both6[i][index]=both6[i][index],4 #4=redirects to smth value
            else:
                index=int(both6[i].index(j))
                new_both6[i][index]=both6[i][index],0 #no page
    return new_both6
new_both6=check_pages_in_list(both6)
new_both6_=check_pages_in_list(substr_wiki_m)

In [24]:
l="fr"
with open(l+'/wiki/berufe_levenshtein.json', 'w') as f: #m_no_page:[wiki_pages_in_beruf_category]
    json.dump(new_both6,  f,  indent=4)#,ensure_ascii=False)
with open(l+'/wiki/berufe_substring.json', 'w') as f: #m_no_page:[wiki_pages_in_beruf_category_withsubstring_inclusion]
    json.dump(new_both6_,  f,  indent=4)#,ensure_ascii=False)
    

In [39]:
for i in new_both6_:
    print i
    for j in new_both6_[i]:
        print j[0],j[1]
    print "---------------"
for i in new_both6:
    print i
    for j in new_both6[i]:
        print j[0],j[1]
    print "---------------"

Monteur - Électricien
Électricien 1
Monteur 0
---------------
Chauffeur de voiture de tourisme
Chauffeur 0
---------------
Inspecteur des matériels de pulvérisation
Inspecteur 0
---------------
Technicien en réseau de communication
Technicien 0
---------------
Menuisier-Agenceur
Menuisier 0
---------------
Chocolatier confiseur
Chocolatier 0
---------------
Technicien de laboratoire médical
Technicien de laboratoire 0
Technicien 0
---------------
Teinturier pressing
Teinturier 0
---------------
Officier chargé du quart à la passerelle
Officier chargé du quart 0
---------------
Carreleur - Mosaïste
Mosaïste 1
Carreleur 0
---------------
Photographe navigant professionnel
Photographe 1
---------------
Bijoutier Joaillier
Bijoutier 0
---------------
Expert-foncier, agricole et forestier
Expert 0
---------------
Plâtrier-plaquiste
Plâtrier 0
---------------
Serrurier - Métallier
Serrurier 4
---------------
Staffeur - Stucateur
Stucateur 0
---------------
Enseignant de la conduite et de la 

In [ ]:
#do something manually with list of levenstein proff
#######

# new table

In [23]:
import json
from prettytable import PrettyTable

def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)


l="fr"
m_no_page=load_simple_json(l+'/wiki/m_no_page.json')
m_redirection=load_simple_json(l+'/wiki/m_redirection.json')
m_page=load_simple_json(l+'/wiki/m_page.json')

#m_page_levenstein==load_simple_json(l+'/wiki/m_page_levenstein.json')
#n_page_levenshtein=load_simple_json(l+'/wiki/n_page_levenshtein.json')



words=len(m_no_page)+len(m_page)+ len(m_redirection)


t = PrettyTable(["--{0}--".format(l),"Occup.", 'Wiki pages', "Redirects","No wiki page or redir"])
            #,"Ambigious pages"
t.add_row(['Masculine',words, len(m_page),  
           len(m_redirection),len(m_no_page)]) #,len(page_m_ambigious)

print t


+-----------+--------+------------+-----------+-----------------------+
|   --fr--  | Occup. | Wiki pages | Redirects | No wiki page or redir |
+-----------+--------+------------+-----------+-----------------------+
| Masculine |  252   |     92     |     47    |          113          |
+-----------+--------+------------+-----------+-----------------------+
